# 라마 인덱스 기본 설치

In [1]:
!pip install -q openai
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.5/966.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s 

In [2]:
# 절차 확인차, 로그 레벨을 낮춰서 확인
import logging
import sys

# 내부적으로 작동하는 로그도 모두 출력하여 노트에서 볼 수 있게 수정
logging.basicConfig( stream=sys.stdout, level= logging.DEBUG, force= True)

# 라마 인덱스로 만든 질의응답

In [3]:
import llama_index
import openai

llama_index.__version__, openai.__version__

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/vocab.bpe HTTP/1.1" 200 456318
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/encoder.json HTTP/1.1" 200 1042301


('0.9.15.post2', '1.4.0')

## 문서로드 -> Document 객체들 ; 산출물

In [4]:
from llama_index import SimpleDirectoryReader
from llama_index import Document
from llama_index import GPTVectorStoreIndex
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 3.3 MB/s eta 0:00:00


### 특정 폴더 밑에 파일 로드

In [5]:
doc_dir = '/content/drive/MyDrive/새싹_파이널프로젝트/eraseplease_test'
documents = SimpleDirectoryReader( doc_dir ).load_data()
# Total files added: 7    <- 파일 로드 확인

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 5


ImportError: ignored

In [ ]:
documents[0]

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/새싹_파이널프로젝트/data/아산병원 최종본_1.xlsx")

In [ ]:
len(df)

1633

In [ ]:
df.head(5)

,질병명,진료과,증상,관련질환,동의어,부위,출처
0,PSA 상승(Elevated PSA),비뇨의학과,무증상,"전립선암,양성 전립선 비대증,만성 전립선염",전립선특이항원 상승,골반,아산병원
1,가진통(False labor),산부인과,"비정상적인 진통,불규칙적 진통","조산,조기 진통,만기 정상 분만",NaN,골반,아산병원
2,간질성 방광염(Interstitial Cystitis),비뇨의학과,"빈뇨,야간뇨,긴박뇨,방광 팽만,복부 통증","골반 염증성 질환,급성 방광염","만성 방광염,방광 통증 증후군",골반,아산병원
3,강직성 척추염(Ankylosing spondylitis),"재활의학과,류마티스내과","요통,관절의 경직,엉덩이 통증,골반 통증,말초 통증,어깨의 통증,관절통","퇴행성 척추 장애,포도막염,척추병증,요추관 협착증","경직성 척추염,강직척추염",골반,아산병원
4,계류 유산(Missed abortion),산부인과,"복부 통증,커지지 않는 자궁,멍,비정상적 질출혈,비출혈,유방의 퇴행,체중감소",유산,NaN,골반,아산병원


In [ ]:
df_filter = df.drop(labels = '출처', axis= 1)

In [ ]:
df_filter.head(2)

,질병명,진료과,증상,관련질환,동의어,부위
0,PSA 상승(Elevated PSA),비뇨의학과,무증상,"전립선암,양성 전립선 비대증,만성 전립선염",전립선특이항원 상승,골반
1,가진통(False labor),산부인과,"비정상적인 진통,불규칙적 진통","조산,조기 진통,만기 정상 분만",NaN,골반


In [ ]:
len(df_filter)

1633

In [ ]:
df_filter=df_filter.drop_duplicates(subset = "질병명")

In [ ]:
#df_filter=df_filter[~df_filter['질병명'].duplicated()]

In [ ]:
#df_filter_du = df_filter[df_filter['질병명'].duplicated()]

In [ ]:
len(df_filter)

1273

In [ ]:
df_filter

,질병명,진료과,증상,관련질환,동의어,부위
0,PSA 상승(Elevated PSA),비뇨의학과,무증상,"전립선암,양성 전립선 비대증,만성 전립선염",전립선특이항원 상승,골반
1,가진통(False labor),산부인과,"비정상적인 진통,불규칙적 진통","조산,조기 진통,만기 정상 분만",NaN,골반
2,간질성 방광염(Interstitial Cystitis),비뇨의학과,"빈뇨,야간뇨,긴박뇨,방광 팽만,복부 통증","골반 염증성 질환,급성 방광염","만성 방광염,방광 통증 증후군",골반
3,강직성 척추염(Ankylosing spondylitis),"재활의학과,류마티스내과","요통,관절의 경직,엉덩이 통증,골반 통증,말초 통증,어깨의 통증,관절통","퇴행성 척추 장애,포도막염,척추병증,요추관 협착증","경직성 척추염,강직척추염",골반
4,계류 유산(Missed abortion),산부인과,"복부 통증,커지지 않는 자궁,멍,비정상적 질출혈,비출혈,유방의 퇴행,체중감소",유산,NaN,골반
...,...,...,...,...,...,...
1624,흉막 삼출(Pleural effusion),호흡기내과,"폐기능저하,기침,빈호흡,호흡곤란,가슴 통증","폐렴,소세포성 폐암,비소세포성 폐암,폐암,폐 혈전색전증,폐결핵",흉수,가슴
1625,흉부 대동맥류(Thoracic aortic Aneurysm),"심장내과,심장병원","어깨의 통증,기침,목 주변 부종,목의 통증,성대 이상,요통","복부 대동맥류,죽상경화증,대동맥판막 기능부전",가슴대동맥류,가슴
1626,흉선암(Thymus Cancer),"종양내과,방사선종양학과,암병원,심장혈관흉부외과","근력 약화,가슴 통증,삼키기 곤란,어지러움,무증상,두통,기침,호흡곤란","자가면역 질환,림프종,호지킨 림프종,비호지킨 림프종,중증 근무력증","가슴샘종,흉선암종,흉선종",가슴
1627,흉통(Chest pain),"심장병원,응급의학과,심장내과,노년내과",가슴 통증,"협심증,대동맥판막 기능부전,불안정형 협심증,오래된 심근경색증,급성 심근경색증,급성 ...",가슴통증,가슴


In [ ]:
df_filter['질병명'] = df_filter['질병명'].drop_duplicates()
df_filter.head(5)
len(df_filter)

1633

In [ ]:
df_filter['질병명'].duplicated().sum()

359

In [ ]:
df_filter.head(5)

,질병명,진료과,증상,관련질환,동의어,부위
0,PSA 상승(Elevated PSA),비뇨의학과,무증상,"전립선암,양성 전립선 비대증,만성 전립선염",전립선특이항원 상승,골반
1,가진통(False labor),산부인과,"비정상적인 진통,불규칙적 진통","조산,조기 진통,만기 정상 분만",NaN,골반
2,간질성 방광염(Interstitial Cystitis),비뇨의학과,"빈뇨,야간뇨,긴박뇨,방광 팽만,복부 통증","골반 염증성 질환,급성 방광염","만성 방광염,방광 통증 증후군",골반
3,강직성 척추염(Ankylosing spondylitis),"재활의학과,류마티스내과","요통,관절의 경직,엉덩이 통증,골반 통증,말초 통증,어깨의 통증,관절통","퇴행성 척추 장애,포도막염,척추병증,요추관 협착증","경직성 척추염,강직척추염",골반
4,계류 유산(Missed abortion),산부인과,"복부 통증,커지지 않는 자궁,멍,비정상적 질출혈,비출혈,유방의 퇴행,체중감소",유산,NaN,골반


# faiss 파이스 - 유사도 검사

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00


### 모듈 가져오기

In [ ]:
import openai

openai.__version__

'1.3.5'

### API KEY 세팅

In [ ]:
import os
from google.colab import userdata


# 운영체계 상에 환경변수 설정 -> api 사용시 무조건 참조를 한다
API_KEY_NAME = 'OPENAI_API_KEY'
os.environ[API_KEY_NAME] = userdata.get(API_KEY_NAME)

## API를 사용하는 메인 객체 획득

In [ ]:
from openai import OpenAI, AsyncOpenAI # 동기식, 비동기식(AsyncOpenAI) 처리하는 객체
import json

# 키를 세팅해서 api를 사용하는 메인 객체 획득
client = OpenAI(api_key = os.environ.get(API_KEY_NAME))

In [ ]:
input_text = '목이 아프고 열이 나.'

def gpt_embedding_sentences( text ):
  response = client.embeddings.create(
      model = 'text-embedding-ada-002',
      input = text
  )
  return response.data

input_text_vec = [sent.embedding for sent in gpt_embedding_sentences( input_text ) ]

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7bc8913eb880>, 'json_data': {'input': '목이 아프고 열이 나.', 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7bc891408df0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7bc89147c040> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7bc89140a3e0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']

In [ ]:
import numpy as np

input_embeds = np.array( input_text_vec ) .astype('float32')
print (input_embeds.shape)

(1, 1536)


In [ ]:
target_texts = [
    '목이 아프고 열이나면 어떤 질환이 예상되나요?'
    '자궁경부암은 어떤 진료과에 가야 하나요?'
    '근육 경련이 있고 어지러우면 어떤 진료과에 가야하고 어떤 질환이 의심돼?'
]

target_embeddings = [sent.embedding for sent in gpt_embedding_sentences( target_texts )]
target_embeddings = np.array( target_embeddings ).astype('float32')
target_embeddings.shape

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7bc891162ef0>, 'json_data': {'input': ['목이 아프고 열이나면 어떤 질환이 예상되나요?자궁경부암은 어떤 진료과에 가야 하나요?근육 경련이 있고 어지러우면 어떤 진료과에 가야하고 어떤 질환이 의심돼?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7bc8911ec460>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7bc89147c040> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7bc8911eca90>
DEBUG:httpcore.http11:send_request_headers.star

(1, 1536)

In [ ]:
import faiss
index_db = faiss.IndexFlatL2(1536) # 문장 1개에 대한 토큰 최대수 입력 - 인덱스 디비 세팅

index_db.add( target_embeddings )

In [ ]:
# 유사도 검색 수행
# D: 가장 가까운 벡터와의 거리
# I: 가장 가까운 벡터의 인덱스
D, I = index_db.search( input_embeds, 1 )
print(D)
print(I)

[[0.28514272]]
[[0]]


In [ ]:
input_text, target_texts[I[0][0]]

('목이 아프고 열이 나.',
 '목이 아프고 열이나면 어떤 질환이 예상되나요?자궁경부암은 어떤 진료과에 가야 하나요?근육 경련이 있고 어지러우면 어떤 진료과에 가야하고 어떤 질환이 의심돼?')

# chroma db - 홈페이지 참고
- https://breezymind.com/vector-db-openai-response-optimize/

### 모듈 가져오기

In [ ]:
import openai

openai.__version__

'1.3.5'

### API KEY 세팅

In [ ]:
import os
from google.colab import userdata


# 운영체계 상에 환경변수 설정 -> api 사용시 무조건 참조를 한다
API_KEY_NAME = 'OPENAI_API_KEY'
os.environ[API_KEY_NAME] = userdata.get(API_KEY_NAME)

## API를 사용하는 메인 객체 획득

In [ ]:
from openai import OpenAI, AsyncOpenAI # 동기식, 비동기식(AsyncOpenAI) 처리하는 객체
import json

# 키를 세팅해서 api를 사용하는 메인 객체 획득
client = OpenAI(api_key = os.environ.get(API_KEY_NAME))

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32

In [ ]:
import chromadb

In [ ]:
client = chromadb.PersistentClient(path = "../data")

In [ ]:
collection = client.get_or_create_collection(
    name = "illness-specialty",
    metadata = {"hnsw:space":"cosine"}
)

In [ ]:
for idx in range(len(df_filter)):
  item = df_filter.iloc[idx]
  print(item)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
부위                                                      배
Name: 634, dtype: object
질병명                        부신 기능부전(Adrenal insufficiency)
진료과                                   내분비내과,신생아과,소아내분비대사과
증상      구토,식욕부진,권태감,피로감,체중감소,저혈당,의식 저하,색소 침착,오심,복부 통증,...
관련질환    곰팡이 감염,뇌하수체 기능부전,뇌하수체 양성 및 악성 신생물,쿠싱 증후군,부신 종양...
동의어          부신기능부전증,부신피질기능부전,부신피질기능저하증,애디슨병,에디슨병,부신기능저하증
부위                                                      배
Name: 635, dtype: object
질병명                                  부신 종양(Adrenal tumor)
진료과                                           비뇨의학과,내분비내과
증상      남성의 여성형 유방,기운없음,발한,체중감소,당뇨,가슴 두근거림,남성스러운 목소리,복...
관련질환                          당뇨병,부신 기능부전,쿠싱 증후군,부신암,심계항진
동의어     Adrenal gland tumor,부신선종,부신양성종양,부신종,부신피질 세포 양성...
부위                                                      배
Name: 636, dtype: object
질병명                             부신암(Adrenal gland cancer)
진료과                                            종양내과,내분비내과
증상      여드름,복부 불편감,

In [ ]:
df_filter

,질병명,진료과,증상,관련질환,동의어,부위
0,PSA 상승(Elevated PSA),비뇨의학과,무증상,"전립선암,양성 전립선 비대증,만성 전립선염",전립선특이항원 상승,골반
1,가진통(False labor),산부인과,"비정상적인 진통,불규칙적 진통","조산,조기 진통,만기 정상 분만",NaN,골반
2,간질성 방광염(Interstitial Cystitis),비뇨의학과,"빈뇨,야간뇨,긴박뇨,방광 팽만,복부 통증","골반 염증성 질환,급성 방광염","만성 방광염,방광 통증 증후군",골반
3,강직성 척추염(Ankylosing spondylitis),"재활의학과,류마티스내과","요통,관절의 경직,엉덩이 통증,골반 통증,말초 통증,어깨의 통증,관절통","퇴행성 척추 장애,포도막염,척추병증,요추관 협착증","경직성 척추염,강직척추염",골반
4,계류 유산(Missed abortion),산부인과,"복부 통증,커지지 않는 자궁,멍,비정상적 질출혈,비출혈,유방의 퇴행,체중감소",유산,NaN,골반
...,...,...,...,...,...,...
1624,흉막 삼출(Pleural effusion),호흡기내과,"폐기능저하,기침,빈호흡,호흡곤란,가슴 통증","폐렴,소세포성 폐암,비소세포성 폐암,폐암,폐 혈전색전증,폐결핵",흉수,가슴
1625,흉부 대동맥류(Thoracic aortic Aneurysm),"심장내과,심장병원","어깨의 통증,기침,목 주변 부종,목의 통증,성대 이상,요통","복부 대동맥류,죽상경화증,대동맥판막 기능부전",가슴대동맥류,가슴
1626,흉선암(Thymus Cancer),"종양내과,방사선종양학과,암병원,심장혈관흉부외과","근력 약화,가슴 통증,삼키기 곤란,어지러움,무증상,두통,기침,호흡곤란","자가면역 질환,림프종,호지킨 림프종,비호지킨 림프종,중증 근무력증","가슴샘종,흉선암종,흉선종",가슴
1627,흉통(Chest pain),"심장병원,응급의학과,심장내과,노년내과",가슴 통증,"협심증,대동맥판막 기능부전,불안정형 협심증,오래된 심근경색증,급성 심근경색증,급성 ...",가슴통증,가슴


In [ ]:
# 데이터 준비
# 인덱스
indexs = []
# 메타데이터
docu_meta = []
documents = []

for idx in range(len(df_filter)):
    item = df_filter.iloc[idx]
    id = item['질병명']
    document = f"{item['질병명']}: {item['진료과']} : {item['증상']}"
    meta = {
      "specialty" : item['진료과']
    }
    indexs.append(id)
    docu_meta.append(meta)
    documents.append(document)


In [ ]:
# for idx in range(len(df_filter)):
#     print(f"Debug: idx={idx}")
#     item = df_filter.iloc[idx]
#     print(f"Debug: item={item}")
#     id = item
#     print(f"Debug: id={id}")
#     document = f"{item['질병명']}: {item['진료과']} : {item['증상']}"
#     print(f"Debug: document={document}")
#     meta = {
#       "specialty" : item['진료과']
#     }
#     print(f"Debug: meta={meta}")
#     indexs.append(id)
#     docu_meta.append(meta)
#     documents.append(document)


In [ ]:
# DB 저장
collection.add(
    documents = documents,
    metadatas = docu_meta,
    ids = indexs
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 91.8MiB/s]


In [ ]:
# DB 쿼리
collection.query(
    query_texts = ["목이 붓고 갑자기 살이 빠지면 어떤 질병이야?"],
    #query_texts = ["배가 아프면 어떤 진료과를 찾아가야해?"],
    #query_texts = ["호흡곤란,소변량 감소"],
    n_results = 3
)

{'ids': [['태열()', '식도염(Esophagitis)', '수정대혈관 전위(Corrected TGA)']],
 'distances': [[0.37287580966949463, 0.44254571199417114, 0.4659093230896527]],
 'metadatas': [[{'specialty': '알레르기내과,피부과'},
   {'specialty': '노년내과,소화기내과'},
   {'specialty': '소아심장외과'}]],
 'embeddings': None,
 'documents': [['태열(): 알레르기내과,피부과 : 두드러기,피부 건조,물집,딱지,피부소양감',
   '식도염(Esophagitis): 노년내과,소화기내과 : 구토,오한,작열감,토혈,열,오심,흑색변,삼키기 곤란',
   '수정대혈관 전위(Corrected TGA): 소아심장외과 : 빈맥,가슴 두근거림,호흡곤란,수유 곤란,청색증,발한,발육부전']],
 'uris': None,
 'data': None}

In [ ]:
#pip openai migrate

ERROR: unknown command "openai"


In [ ]:
#pip install openai == 0.28

ERROR: Invalid requirement: '=='


In [ ]:
import openai

openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role":"ai", "content": "전문적인 의사"},
        {"role": "user", "content" : "살이 갑자기 빠지면 어떤 진료과에 가야해?"},
        {"role": "assistant", "content" : "진단을 해주면 안 되고 어떤 질병이 예상된다고 해야해"}
    ]
)

APIRemovedInV1: ignored

In [ ]:
# vs code로 실행
# D:\SxS\파이널프로젝트\'hospital_public.py'